# Divergence

Computing the divergence of vector fields is a fundamental operation in vector calculus with applications in fluid dynamics, electromagnetism, and many other fields. This user-guide notebook showcases how to compute the divergence of a vector field.

In [ ]:
import numpy as np

import uxarray as ux

## Divergence Computation

### Background

The **divergence** of a vector field **V** = (u, v) is a scalar field that measures the "outflow" of the vector field from each point:

$$\nabla \cdot \mathbf{V} = \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}$$

**Physical Interpretation:**
- **Positive divergence**: Indicates a "source" - the vector field is flowing outward from that point
- **Negative divergence**: Indicates a "sink" - the vector field is flowing inward to that point
- **Zero divergence**: Indicates incompressible flow - no net outflow or inflow

### Implementation

In UXarray, the divergence is computed using the existing gradient infrastructure. The method leverages the finite-volume discretization to compute gradients of each vector component and then sums the relevant partial derivatives.

| **Input**                     |    **Usage**                    | **Output**                  |
| ----------------------------- | :-----------------------------: | --------------------------- |
| Vector field (u, v)           | `u.divergence(v)`               | Scalar field $\nabla \cdot \mathbf{V}$ |

## Data

This notebook uses a subset of a 30km MPAS atmosphere grid, taken centered at 45 degrees longitude and 0 degrees latitude with a radius of 2 degrees.
- `face_lon`: Longitude at cell-centers
- `face_lat`: Latitude at cell-centers
- `gaussian`: Gaussian initialized at the center of the grid
- `inverse_gaussian`: Inverse of the gaussian above.

In [ ]:
base_path = "../../test/meshfiles/mpas/dyamond-30km/"
grid_path = base_path + "gradient_grid_subset.nc"
data_path = base_path + "gradient_data_subset.nc"

uxds = ux.open_dataset(grid_path, data_path)
print(f"Grid has {uxds.uxgrid.n_face} faces")
print(f"Available variables: {list(uxds.data_vars.keys())}")
uxds

## Usage

The divergence method is available on `UxDataArray` objects and follows this signature:

```python
div_result = u_component.divergence(v_component)
```

The method returns a `UxDataArray` containing the divergence values with the same shape and grid as the input components.

### Gaussian Fields

In [ ]:
# Use Gaussian fields as vector components
u_gauss = uxds["gaussian"]
v_gauss = uxds["inverse_gaussian"]

# Compute divergence
div_gauss = u_gauss.divergence(v_gauss)

# Handle NaN values from boundary faces
finite_mask = np.isfinite(div_gauss.values)
finite_values = div_gauss.values[finite_mask]

print(f"Total faces: {len(div_gauss.values)}")
print(f"Interior faces: {len(finite_values)}")
print(f"Boundary faces: {np.isnan(div_gauss.values).sum()}")

if len(finite_values) > 0:
    print(
        f"Finite divergence range: [{finite_values.min():.6f}, {finite_values.max():.6f}]"
    )
    print(f"Mean divergence (finite): {finite_values.mean():.6f}")

### Constant Fields (Mathematical Validation)

The divergence of a constant vector field should be zero everywhere (within numerical precision). This provides an important validation of our implementation.

In [ ]:
# The divergence of a constant vector field should be zero everywhere
u_constant = uxds["face_lat"] * 0 + 1.0  # Constant = 1
v_constant = uxds["face_lat"] * 0 + 2.0  # Constant = 2

# Compute divergence
div_constant = u_constant.divergence(v_constant)

# Handle NaN values from boundary faces
finite_mask = np.isfinite(div_constant.values)
finite_values = div_constant.values[finite_mask]

print(f"Total faces: {len(div_constant.values)}")
print(f"Finite values: {len(finite_values)}")
print(f"NaN values (boundary faces): {np.isnan(div_constant.values).sum()}")

if len(finite_values) > 0:
    print(
        f"Finite divergence range: [{finite_values.min():.10f}, {finite_values.max():.10f}]"
    )
    print(f"Maximum absolute divergence (finite): {np.abs(finite_values).max():.2e}")
    print(f"Mean absolute divergence (finite): {np.abs(finite_values).mean():.2e}")

    # Should be close to zero for constant field
    max_abs_div = np.abs(finite_values).max()
    if max_abs_div < 1e-10:
        print("✓ Divergence is approximately zero as expected")
    else:
        print(f"⚠ Divergence is {max_abs_div:.2e} (may be due to discretization)")

## Computing the Laplacian

The Laplacian of a scalar field can be computed as the divergence of its gradient: $\nabla^2 \phi = \nabla \cdot (\nabla \phi)$

This demonstrates the power of combining UXarray's vector calculus operations.

In [ ]:
# Compute gradient of the Gaussian field
grad_gaussian = uxds["gaussian"].gradient()

# Extract gradient components
grad_x = grad_gaussian["zonal_gradient"]
grad_y = grad_gaussian["meridional_gradient"]

# Compute Laplacian as divergence of gradient
laplacian = grad_x.divergence(grad_y)

# Analyze results
finite_mask = np.isfinite(laplacian.values)
finite_laplacian = laplacian.values[finite_mask]

print("Laplacian computed successfully!")
print(f"Interior faces: {len(finite_laplacian)}")
print(f"Boundary faces: {np.isnan(laplacian.values).sum()}")

if len(finite_laplacian) > 0:
    print(
        f"Laplacian range: [{finite_laplacian.min():.6f}, {finite_laplacian.max():.6f}]"
    )
    print(f"Mean Laplacian: {finite_laplacian.mean():.6f}")